# Magenta RT: Streaming music generation!

<a href="https://colab.research.google.com/github/magenta/magenta-realtime/blob/main/notebooks/Magenta_RT_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Magenta RealTime is a Python library for streaming music audio generation on
your local device. It is the open weights / on device companion to
[MusicFX DJ Mode](https://labs.google/fx/tools/music-fx-dj) and the
[Lyria RealTime API](https://ai.google.dev/gemini-api/docs/music-generation).

-   [Blog Post](https://g.co/magenta/rt)
-   [Repository](https://github.com/magenta/magenta-realtime)
-   [HuggingFace](https://huggingface.co/google/magenta-realtime)

### Generating audio with Magenta RT

Magenta RT generates audio in short chunks (2s) given a finite amount of past
context (10s). We use crossfading to mitigate boundary artifacts between chunks.
More details on our model are coming soon in a technical report!

![Animation of chunk-by-chunk generation in Magenta RT](https://raw.githubusercontent.com/magenta/magenta-realtime/refs/heads/main/notebooks/diagram.gif)

# Step 1: 😴 One-time setup

In [ ]:
# @title **Run this cell** to install dependencies (~5 minutes)
# @markdown Make sure you are running on **`v2-8 TPU` runtime** via `Runtime > Change Runtime Type`

# @markdown Colab may prompt you to restart session. **Wait until the cell finishes running to restart**!

# Clone library
!git clone https://github.com/magenta/magenta-realtime.git

# Magenta RT requires nightly TF builds, but stable may be installed.
# Force nightly to take precedence by uninstalling and reinstalling.
# Temporary workaround until MusicCoCa supported by TF stable.
_all_tf = 'tensorflow tf-nightly tensorflow-cpu tf-nightly-cpu tensorflow-tpu tf-nightly-tpu tensorflow-hub tf-hub-nightly tensorflow-text tensorflow-text-nightly'
_nightly_tf = 'tf-nightly tensorflow-text-nightly tf-hub-nightly'

# Install library and dependencies
# If running on TPU (recommended, runs on free tier Colab TPUs):
!pip install -e magenta-realtime/[tpu] && pip uninstall -y {_all_tf} && pip install {_nightly_tf}
# Uncomment if running on GPU (requires A100 via Colab Pro):
# !pip install -e magenta-realtime/[gpu] && pip uninstall -y {_all_tf} && pip install {_nightly_tf}

In [ ]:
# @title **Restart session (`Runtime > Restart Session`) and run this cell** to initialize model (~5 minutes)

from magenta_rt import system

# Fetch checkpoints and initialize model (may take up to 5 minutes)
MRT = system.MagentaRT(
    tag="large", device="tpu:v2-8", skip_cache=True, lazy=False
)

# Step 2: 🤘 Streaming music generation! 🎵

**Run the cell below and click the `start` button to begin streaming!**

**Instructions**. Type in text prompts to control the overall style of the generated music in real time. The sliders by the prompts change the influence of each text prompt on the overall output. The other controls change various aspects of the system behavior (expand below for additional information).

**Disclaimer**. Magenta RT's training data primarily consists of Western instrumental music. As a consequence, Magenta RT has incomplete coverage of both vocal performance and the broader landscape of rich musical traditions worldwide. For real-time generation with broader style coverage, we refer users to our [Lyria RealTime API](g.co/magenta/lyria-realtime). See our [model card](https://huggingface.co/google/magenta-realtime) for more information.

<details>
  <summary>Click to expand for additional information on the controls</summary>

*   **buffering_amount_seconds**: Increase this value if you experience audio
    drops during generation. This will come at the expense of a greater latency,
    but might help with internet connection issues. *You need to relaunch the
    cell if you choose to modify this value*.

*   **sampling options**

    *   **temperature**: This controls how *chaotic* the model behaves. Low
        temperature values (e.g., 0.9) will make the model's choices more
        predictable and stable. High values (e.g., 1.5) will encourage more
        surprising and experimental musical ideas, but can also lead to
        instability.

    *   **topk**: This parameter filters the model's vocabulary at each step. It
        forces the model to choose its next prediction only from the *k* most
        likely options.

        *   A **low `topk`** value (e.g., 40) restricts the model to a smaller,
            safer palette of options. This leads to more coherent and
            predictable music that is less likely to have dissonant errors, but
            can sometimes feel repetitive.
        *   A **high `topk`** value gives the model a much wider range of
            choices, allowing for more variety and unexpected turns. This can
            make the output more creative, but also noisier.

    *   **guidance**: This controls how strictly the generated music should
        adhere to the **text prompts**.

        *   A **higher value** will push the model to produce a textbook example
            of the chosen style, emphasizing its key characteristics.
        *   A **lower value** will treat the text prompts more as a loose
            inspiration, allowing the model more creative freedom to deviate and
            blend other influences.

*   **Reset**: stop audio, and resets the model.

*   **Text prompts**: Next to each text prompt is a slider that controls how
    much each prompt should be affecting the model. This allows the creation of
    *mixed* embeddings (try mixing synthwave and flamenco guitar together !).
    You can also type your own prompt and modify existing ones.
</details>

In [ ]:
# @title Run this cell to start demo

import concurrent.futures
import functools
from typing import Any, Sequence

import IPython.display as ipd
import ipywidgets as ipw
import numpy as np

from magenta_rt import system
from magenta_rt.colab import utils
from magenta_rt.colab import widgets

buffering_amount_seconds = 0 # @param {"type":"slider","min":0,"max":4,"step":0.1}
buffering_amount_samples = int(np.ceil(buffering_amount_seconds * 48000))


class AudioFade:
  """Handles the cross fade between audio chunks.

  Args:
    chunk_size: Number of audio samples per predicted frame (current
      SpectroStream models produces 25Hz frames corresponding to 1920 audio
      samples at 48kHz)
    num_chunks: Number of audio chunks to fade between.
    stereo: Whether the predicted audio is stereo or mono.
  """

  def __init__(self, chunk_size: int, num_chunks: int, stereo: bool):
    fade_size = chunk_size * num_chunks
    self.fade_size = fade_size
    self.num_chunks = num_chunks

    self.previous_chunk = np.zeros(fade_size)
    self.ramp = np.sin(np.linspace(0, np.pi / 2, fade_size)) ** 2

    if stereo:
      self.previous_chunk = self.previous_chunk[:, np.newaxis]
      self.ramp = self.ramp[:, np.newaxis]

  def reset(self):
    self.previous_chunk = np.zeros_like(self.previous_chunk)

  def __call__(self, chunk: np.ndarray) -> np.ndarray:
    chunk[: self.fade_size] *= self.ramp
    chunk[: self.fade_size] += self.previous_chunk
    self.previous_chunk = chunk[-self.fade_size :] * np.flip(self.ramp)
    return chunk[: -self.fade_size]


class MagentaRTStreamer:
  """Audio streamer class.

  This class holds a pretrained Magenta RT model, a cross fade state, a
  generation state and an asynchronous executor to handle the embedding of text
  prompt without interrupting the audio thread.

  Args:
    system: A MagentaRTBase instance.
  """

  def __init__(self, system: system.MagentaRTBase):
    self.system = system
    self.fade = AudioFade(chunk_size=1920, num_chunks=1, stereo=True)
    self.state = None
    self.executor = concurrent.futures.ThreadPoolExecutor()
    self.audio_streamer = None

  @functools.cache
  def embed_style(self, style: str):
    return self.executor.submit(self.system.embed_style, style)

  def get_style_embedding(
      self, params: dict[str, Any], force_wait: bool = False
  ):
    num_prompts = sum(map(lambda s: "prompt" in s, params.keys()))

    weighted_embedding = np.zeros((768,), dtype=np.float32)
    total_weight = 0.0

    for i in range(num_prompts):
      weight = params[f"prompt_{i}"]
      if not weight:
        continue
      text = params[f"style_{i}"]
      embedding = self.embed_style(text)

      if force_wait:
        embedding.result()

      if embedding.done():
        weighted_embedding += embedding.result() * weight
        total_weight += weight

    if total_weight:
      weighted_embedding /= total_weight

    return weighted_embedding

  def __call__(self, inputs):
    del inputs
    ui_params = utils.Parameters.get_values()

    chunk, self.state = self.system.generate_chunk(
        state=self.state,
        style=self.get_style_embedding(ui_params),
        seed=None,
        **ui_params,
    )
    chunk = self.fade(chunk.samples)
    return chunk

  def preflight(self):
    ui_params = utils.Parameters.get_values()
    self.get_style_embedding(ui_params, force_wait=False)
    self.get_style_embedding(ui_params, force_wait=True)
    self.audio_streamer.reset_ring_buffer()

  def reset(self):
    self.state = None
    self.fade.reset()
    if self.audio_streamer is not None:
      self.audio_streamer.reset_ring_buffer()

  def start(self):
    self.audio_streamer = utils.AudioStreamer(
        self,
        rate=48000,
        buffer_size=48000 * 2,
        warmup=True,
        num_output_channels=2,
        additional_buffered_samples=buffering_amount_samples,
        start_streaming_callback=self.preflight,
    )
    self.reset()


# BUILD UI


def build_prompt_ui(default_prompts: Sequence[str]):
  """Add interactive prompt widgets and register them."""
  prompts = []

  for p in default_prompts:
    prompts.append(widgets.Prompt())
    prompts[-1].text.value = p

  prompts[0].slider.value = 1.0

  utils.Parameters.register_ui_elements(
      display=False,
      **{f"prompt_{i}": p.slider for i, p in enumerate(prompts)},
      **{f"style_{i}": p.text for i, p in enumerate(prompts)},
  )
  return [p.get_widget() for p in prompts]


def build_sampling_option_ui():
  """Add interactive sampling option widgets and register them."""
  options = {
      "temperature": ipw.FloatSlider(
          min=0.0,
          max=4.0,
          step=0.01,
          value=1.3,
          description="temperature",
      ),
      "topk": ipw.IntSlider(
          min=0,
          max=1024,
          step=1,
          value=40,
          description="topk",
      ),
      "guidance_weight": ipw.FloatSlider(
          min=0.0,
          max=10.0,
          step=0.01,
          value=5.0,
          description="guidance",
      ),
  }

  utils.Parameters.register_ui_elements(display=False, **options)

  return list(options.values())


utils.Parameters.reset()


try:
  MRT
except NameError:
  print("Magenta RT not initialized. Please run the cell above.")


streamer = MagentaRTStreamer(MRT)


def _reset_state(*args, **kwargs):
  del args, kwargs
  streamer.reset()


reset_button = ipw.Button(description="reset")
reset_button.on_click(_reset_state)


# Building interactive UI
ipd.display(
    ipw.VBox([
        widgets.area(
            "sampling options",
            *build_sampling_option_ui(),
            reset_button,
        ),
        widgets.area(
            "prompts",
            *build_prompt_ui([
                "synthwave",
                "flamenco guitar",
                "",
                "",
            ]),
        ),
    ])
)

streamer.start()

# Step 3: Understand what is happening behind the hood

Let's start by generating a short (2s) chunk of synthwave

In [ ]:
import IPython.display as ipd

try:
  model = MRT
except NameError:
  model = system.MagentaRT(
      tag="large", device="tpu:v2-8", skip_cache=True, lazy=False
  )

prompt = "synthwave"
embedding = model.embed_style(prompt)

audio, state = model.generate_chunk(
    state=None,
    style=embedding,
    seed=0,
)

ipd.display(ipd.Audio(audio.samples.T, rate=audio.sample_rate))

We can generate longer sequences by concatenating generations while keeping
track of the internal state of the model. We use a crossfade time of 40ms to
concatenate audio chunks as this is the frame length used by SpectroStream when
encoding audio.

In [ ]:
from magenta_rt import audio

num_chunks = 4
state = None
chunks = []

for i in range(num_chunks):
  chunk, state = model.generate_chunk(
      state=state,
      style=embedding,
      seed=i,
  )
  chunks.append(chunk)

concatenated_audio = audio.concatenate(
    chunks,
    crossfade_time=model.config.crossfade_length,
)
ipd.display(
    ipd.Audio(concatenated_audio.samples.T, rate=concatenated_audio.sample_rate)
)

At the core of Magenta RT lies the idea of changing the style embedding *during
generation* to enable smooth transitions between musical concepts. What about
transitioning from "synthwave" to "disco funk" ?

In [ ]:
state = None
chunks = []

styles = [
    "synthwave",
    "disco synthwave",
    "disco",
    "disco funk",
]

for i, style in enumerate(styles):
  chunk, state = model.generate_chunk(
      state=state,
      style=model.embed_style(style),
      seed=i,
      guidance_weight=5.0,
      temperature=1.3,
  )
  chunks.append(chunk)

concatenated_audio = audio.concatenate(
    chunks,
    crossfade_time=model.config.crossfade_length,
)
ipd.display(
    ipd.Audio(concatenated_audio.samples.T, rate=concatenated_audio.sample_rate)
)

A simpler version can be done through the interpolation of musical genres in
*the embedding space*.

In [ ]:
import numpy as np

state = None
chunks = []

embed_a = model.embed_style("synthwave")
embed_b = model.embed_style("disco funk")

weight = np.linspace(0, 1, 8, endpoint=True)

embeddings = embed_a[None] + weight[:, None] * (embed_b - embed_a)
embeddings = embeddings.astype(np.float32)


for i, embedding in enumerate(embeddings):
  chunk, state = model.generate_chunk(
      state=state,
      style=embedding,
      seed=i,
      guidance_weight=5.0,
      temperature=1.3,
  )
  chunks.append(chunk)

concatenated_audio = audio.concatenate(
    chunks,
    crossfade_time=model.config.crossfade_length,
)
ipd.display(
    ipd.Audio(concatenated_audio.samples.T, rate=concatenated_audio.sample_rate)
)

# License and terms

Magenta RealTime is offered under a combination of licenses: the codebase is
licensed under
[Apache 2.0](https://github.com/magenta/magenta-realtime/blob/main/LICENSE), and
the model weights under
[Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/legalcode).

In addition, we specify the following usage terms:

Copyright 2025 Google LLC

Use these materials responsibly and do not generate content, including outputs, that infringe or violate the rights of others, including rights in copyrighted content.

Google claims no rights in outputs you generate using Magenta RealTime. You and your users are solely responsible for outputs and their subsequent uses.

Unless required by applicable law or agreed to in writing, all software and materials distributed here under the Apache 2.0 or CC-BY licenses are distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the licenses for the specific language governing permissions and limitations under those licenses. You are solely responsible for determining the appropriateness of using, reproducing, modifying, performing, displaying or distributing the software and materials, and any outputs, and assume any and all risks associated with your use or distribution of any of the software and materials, and any outputs, and your exercise of rights and permissions under the licenses.